In [1]:
import polars as pl
pl.Config.set_tbl_rows(40)

from datetime import datetime

# Add parent directory to Python path to access project modules
import os, sys
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir) if current_dir.endswith('notebooks') else current_dir

print(f"Project root: {project_root}")
if project_root not in sys.path:
    sys.path.append(project_root)

import numpy as np
from IPython.display import display, HTML

from pricer.pricer_helper import find_vol
from parity_analysis.market_data.deribit_md_manager import DeribitMDManager
from parity_analysis.reporting.html_table_generator import generate_price_comparison_table
from pricer.option_constraints import tighten_option_spread

Project root: /home/user/Python/Baseoffset-Fitting-Manager


In [2]:
def find_bid_ask_vola(df: pl.DataFrame, rate: float, is_call: bool, **kwargs) -> tuple[np.ndarray, np.ndarray]:
    """
    Calculate implied volatilities for bid and ask prices.
    
    Args:
        df: DataFrame with option market data
        rate: Risk-free interest rate
        
    Returns:
        Tuple of (bid_volatilities, ask_volatilities)
    """
    bid_col = 'bp0_C_usd' if is_call else 'bp0_P_usd'
    ask_col = 'ap0_C_usd' if is_call else 'ap0_P_usd'
    
    # Extract data for volatility calculation
    input_data = df[bid_col, ask_col, 'F', 'strike', 'tau']
    
    bid_vola = find_vol(target_value=input_data[:, 0], 
                        F=input_data[:, 2], 
                        K=input_data[:, 3], 
                        T=input_data[:, 4], 
                        r=rate, 
                        option_type='C' if is_call else 'P',
                        kwargs=kwargs)
    
    ask_vola = find_vol(target_value=input_data[:, 1], 
                        F=input_data[:, 2], 
                        K=input_data[:, 3], 
                        T=input_data[:, 4], 
                        r=rate, 
                        option_type='C' if is_call else 'P',
                        kwargs=kwargs)

    return bid_vola, ask_vola


def get_bid_ask_vola(df_option_chain: pl.DataFrame, interest_rate: float, **kwargs) -> pl.DataFrame:
    " get bid and ask implied volatilities for calls and puts, with synthetic arbitrage checks and tightened spreads "

    # tighten the bid-ask spread
    df =\
    df_option_chain.with_columns(
        bp0_C_usd = (pl.col('tightened_bid_price') * pl.col('S')).round(2),
        ap0_C_usd = (pl.col('tightened_ask_price') * pl.col('S')).round(2),
        bp0_P_usd = (pl.col('tightened_bid_price_P') * pl.col('S')).round(2),
        ap0_P_usd = (pl.col('tightened_ask_price_P') * pl.col('S')).round(2),
    ).select(
        ['timestamp','bid_size','tightened_bid_price','bp0_C_usd','ap0_C_usd','tightened_ask_price','ask_size','strike','bid_size_P',
        'tightened_bid_price_P','bp0_P_usd','ap0_P_usd','tightened_ask_price_P','ask_size_P','S','bid_price_fut','expiry','tau']
    ).rename({'bid_size': 'bq0_C', 'tightened_bid_price': 'bp0_C', 'tightened_ask_price': 'ap0_C', 'ask_size': 'aq0_C',
            'bid_size_P': 'bq0_P', 'tightened_bid_price_P': 'bp0_P', 'tightened_ask_price_P': 'ap0_P', 'ask_size_P': 'aq0_P', 'bid_price_fut': 'F'})
    
    # convert from price term into volatility term
    df = df.with_columns(
        r = interest_rate,
        bidVola_C = (pl.Series('bidVola_C', find_bid_ask_vola(df, interest_rate, is_call=True, **kwargs)[0])*100).round(2),
        askVola_C = (pl.Series('askVola_C', find_bid_ask_vola(df, interest_rate, is_call=True, **kwargs)[1])*100).round(2),
        bidVola_P = (pl.Series('bidVola_P', find_bid_ask_vola(df, interest_rate, is_call=False, **kwargs)[0])*100).round(2),
        askVola_P = (pl.Series('askVola_P', find_bid_ask_vola(df, interest_rate, is_call=False, **kwargs)[1])*100).round(2),
    )
    # Check for synthetic arbitrage: ask vol of put >= bid vol of call, and ask vol of call >= bid vol of put
    mask_cross_cp = (df['askVola_P'] < df['bidVola_C'])
    mask_cross_pc = (df['askVola_C'] < df['bidVola_P'])

    if mask_cross_cp.any():
        raise ValueError(
            f"Synthetic violation: ask vol of Put < bid vol of Call at strikes:\n"
            f"{df.filter(mask_cross_cp)[['strike', 'bidVola_C', 'askVola_P']]}"
        )

    if mask_cross_pc.any():
        raise ValueError(
            f"Synthetic violation: ask vol of Call < bid vol of Put at strikes:\n"
            f"{df.filter(mask_cross_pc)[['strike', 'bidVola_P', 'askVola_C']]}"
        )
    return df

In [3]:
# read the data files generated by the baseoffset fitting process
date_str = '20240229'
# find the snapshot market data
snapshot_time = datetime(2024,2,29,0,12)

In [4]:
def get_baseoffset_df(data_str: str) -> pl.DataFrame:
    return pl.read_csv(f'../results/{data_str}/baseoffset_results.csv').with_columns(
        timestamp = pl.col('timestamp').cast(pl.Datetime('ns'))
    )

def get_option_md_df(data_str: str) -> pl.DataFrame:
    df = pl.read_csv(f'../results/{data_str}/conflated_md.csv').with_columns(
        timestamp = pl.col('timestamp').cast(pl.Datetime('ns')),
        expiry_ts = pl.col('expiry_ts').cast(pl.Datetime('ns'))
    ).select(
        ['symbol','timestamp','expiry','strike','bid_size','bid_price','ask_price','ask_size','S','expiry_ts','is_option','is_call','tau']
    ).sort(['timestamp','expiry_ts'])
    print(f"available expiries: {df['expiry'].unique().to_list()}")
    return df

In [5]:
df_baseoffset = get_baseoffset_df(date_str)
df_option_md = get_option_md_df(date_str)

available expiries: ['3MAR24', '15MAR24', '31MAY24', '8MAR24', '26APR24', '27DEC24', '1MAR24', '29MAR24', '22MAR24', '27SEP24', '28JUN24', '29FEB24', '2MAR24']


In [6]:
def get_snapshot_option_chain(option_md_df: pl.DataFrame, baseoffset_df: pl.DataFrame, snapshot_time: datetime) -> pl.DataFrame:
    assert snapshot_time in option_md_df['timestamp'].unique().to_list()
    assert snapshot_time in baseoffset_df['timestamp'].unique().to_list()
    
    df = option_md_df.filter(pl.col('timestamp')==snapshot_time).with_columns(
        option_type = pl.when(pl.col('is_call')).then(pl.lit('C')).otherwise(pl.lit('P'))
    ).select(
        ['symbol','timestamp','expiry','strike','bid_size','bid_price','ask_price','ask_size','S','expiry_ts','option_type','tau']
    ).join(
        baseoffset_df.filter(pl.col('timestamp')==snapshot_time).select(['expiry','timestamp','F','r']),
        on=['timestamp','expiry']
    ).sort(['expiry_ts','strike','option_type'])

    assert df['S'].unique().len() == 1, "S should be unique and constant across all options at the same timestamp"

    return df

In [7]:
df_snapshot_md = get_snapshot_option_chain(df_option_md, df_baseoffset, snapshot_time)

In [8]:
df_snapshot_md.head()

symbol,timestamp,expiry,strike,bid_size,bid_price,ask_price,ask_size,S,expiry_ts,option_type,tau,F,r
str,datetime[ns],str,i64,f64,f64,f64,f64,f64,datetime[ns],str,f64,f64,f64
"""BTC-29FEB24-45000-P""",2024-02-29 00:12:00,"""29FEB24""",45000,null,null,0.0004,13.0,62166.15,2024-02-29 08:00:00,"""P""",0.00089,62184.8,0.4
"""BTC-29FEB24-46000-P""",2024-02-29 00:12:00,"""29FEB24""",46000,null,null,0.0004,13.0,62166.15,2024-02-29 08:00:00,"""P""",0.00089,62184.8,0.4
"""BTC-29FEB24-47000-P""",2024-02-29 00:12:00,"""29FEB24""",47000,null,null,0.0004,13.0,62166.15,2024-02-29 08:00:00,"""P""",0.00089,62184.8,0.4
"""BTC-29FEB24-47500-P""",2024-02-29 00:12:00,"""29FEB24""",47500,null,null,0.0004,13.0,62166.15,2024-02-29 08:00:00,"""P""",0.00089,62184.8,0.4
"""BTC-29FEB24-48000-P""",2024-02-29 00:12:00,"""29FEB24""",48000,null,null,0.0004,13.0,62166.15,2024-02-29 08:00:00,"""P""",0.00089,62184.8,0.4


In [9]:
df_snapshot_md.tail()

symbol,timestamp,expiry,strike,bid_size,bid_price,ask_price,ask_size,S,expiry_ts,option_type,tau,F,r
str,datetime[ns],str,i64,f64,f64,f64,f64,f64,datetime[ns],str,f64,f64,f64
"""BTC-27DEC24-140000-C""",2024-02-29 00:12:00,"""27DEC24""",140000,7.0,0.054,0.0585,6.0,62166.15,2024-12-27 08:00:00,"""C""",0.828288,68876.2,0.1243
"""BTC-27DEC24-140000-P""",2024-02-29 00:12:00,"""27DEC24""",140000,null,null,null,null,62166.15,2024-12-27 08:00:00,"""P""",0.828288,68876.2,0.1243
"""BTC-27DEC24-160000-C""",2024-02-29 00:12:00,"""27DEC24""",160000,7.0,0.039,0.043,7.0,62166.15,2024-12-27 08:00:00,"""C""",0.828288,68876.2,0.1243
"""BTC-27DEC24-180000-C""",2024-02-29 00:12:00,"""27DEC24""",180000,7.0,0.029,0.033,7.0,62166.15,2024-12-27 08:00:00,"""C""",0.828288,68876.2,0.1243
"""BTC-27DEC24-200000-C""",2024-02-29 00:12:00,"""27DEC24""",200000,11.0,0.0225,0.026,10.8,62166.15,2024-12-27 08:00:00,"""C""",0.828288,68876.2,0.1243


In [10]:
# reuse the DeribitMDManager to process the option chain and get the tightening bid-ask spread for backing out the IV
my_expiry = '15MAR24'
assert my_expiry in df_snapshot_md['expiry'].unique().to_list()

df_option_chain =\
DeribitMDManager.get_option_chain(df_snapshot_md.with_columns(
    is_call = pl.col('option_type') == 'C',
    is_put = pl.col('option_type') == 'P',
    bid_price_fut = pl.col('F'),
    ask_price_fut = pl.col('F'),
), my_expiry, snapshot_time)

tightened_option_chain = tighten_option_spread(df_option_chain)

print(f"Comparison of old bid/ask proces and the tightened bid/ask price")

# Generate HTML table using external module
display(HTML(generate_price_comparison_table(tightened_option_chain, table_width="70%", font_size="10px")))

Comparison of old bid/ask proces and the tightened bid/ask price


In [11]:
df_option_with_vola = get_bid_ask_vola(tightened_option_chain, interest_rate=0.1)
df_option_with_vola.head()

timestamp,bq0_C,bp0_C,bp0_C_usd,ap0_C_usd,ap0_C,aq0_C,strike,bq0_P,bp0_P,bp0_P_usd,ap0_P_usd,ap0_P,aq0_P,S,F,expiry,tau,r,bidVola_C,askVola_C,bidVola_P,askVola_P
datetime[ns],f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64
2024-02-29 00:12:00,2.0,0.281,17468.69,17748.44,0.2855,61.0,45000,19.9,0.0013,80.82,124.33,0.002,7.8,62166.15,62665.47,"""15MAR24""",0.041986,0.1,2.36,91.88,81.92,88.08
2024-02-29 00:12:00,2.0,0.2655,16505.11,16784.86,0.27,60.9,46000,42.2,0.0016,99.47,142.98,0.0023,12.8,62166.15,62665.47,"""15MAR24""",0.041986,0.1,2.36,89.9,79.9,85.21
2024-02-29 00:12:00,2.0,0.25,15541.54,15821.29,0.2545,60.9,47000,16.2,0.002,124.33,167.85,0.0027,13.1,62166.15,62665.47,"""15MAR24""",0.041986,0.1,2.36,87.44,78.15,82.68
2024-02-29 00:12:00,2.0,0.2345,14577.96,14857.71,0.239,60.9,48000,41.9,0.0024,149.2,198.93,0.0032,12.3,62166.15,62665.47,"""15MAR24""",0.041986,0.1,1.0,84.61,75.85,80.33
2024-02-29 00:12:00,2.0,0.219,13614.39,13894.13,0.2235,60.9,49000,41.7,0.003,186.5,236.23,0.0038,12.5,62166.15,62665.47,"""15MAR24""",0.041986,0.1,45.15,81.47,74.2,78.04


In [12]:
import plotly.graph_objects as go

fig = go.Figure()
# Plot with error bars for bid/ask implied volatility
fig.add_trace(go.Scatter(
    x=df_option_with_vola['strike'],
    y=(df_option_with_vola['bidVola_C']+df_option_with_vola['askVola_C'])/2,
    error_y=dict(
        type='data',
        array=(df_option_with_vola['askVola_C'] - df_option_with_vola['bidVola_C']).abs()/2,
        visible=True,
        color='blue'
    ),
    mode='markers',
    name='Call IV (Bid/Ask Error Bar)',
    marker=dict(color='blue', symbol='circle')
))
fig.add_trace(go.Scatter(
    x=df_option_with_vola['strike'],
    y=(df_option_with_vola['bidVola_P']+df_option_with_vola['askVola_P'])/2,
    error_y=dict(
        type='data',
        array=(df_option_with_vola['askVola_P'] - df_option_with_vola['bidVola_P']).abs()/2,
        visible=True,
        color='orange'
    ),
    mode='markers',
    name='Put IV (Bid/Ask Error Bar)',
    marker=dict(color='orange', symbol='circle')
))

# fig.add_trace(go.Scatter(x=df_option_with_vola['strike'], y=df_option_with_vola['bidVola_P'], mode='markers', name='Put Bid Vola', marker=dict(color='orange', symbol='triangle-up'))) 
# fig.add_trace(go.Scatter(x=df_option_with_vola['strike'], y=df_option_with_vola['askVola_P'], mode='markers', name='Put Ask Vola', marker=dict(color='orange', symbol='triangle-down')))

fig.add_vline(x=df_option_with_vola['S'][0], line=dict(color='green', dash='dash'), name='Spot Price (S)')

fig.update_layout(
    title=f"Implied Volatility on DERIBIT BTC option at {snapshot_time.strftime('%Y-%m-%d %H:%M')} for Expiry {my_expiry}",
    xaxis_title="Strike Price",
    yaxis_title="Implied Volatility (%)",
    legend_title="Legend",
    template="plotly_white"
)
fig.show()


In [13]:
df_option_with_vola

timestamp,bq0_C,bp0_C,bp0_C_usd,ap0_C_usd,ap0_C,aq0_C,strike,bq0_P,bp0_P,bp0_P_usd,ap0_P_usd,ap0_P,aq0_P,S,F,expiry,tau,r,bidVola_C,askVola_C,bidVola_P,askVola_P
datetime[ns],f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64
2024-02-29 00:12:00,2.0,0.281,17468.69,17748.44,0.2855,61.0,45000,19.9,0.0013,80.82,124.33,0.002,7.8,62166.15,62665.47,"""15MAR24""",0.041986,0.1,2.36,91.88,81.92,88.08
2024-02-29 00:12:00,2.0,0.2655,16505.11,16784.86,0.27,60.9,46000,42.2,0.0016,99.47,142.98,0.0023,12.8,62166.15,62665.47,"""15MAR24""",0.041986,0.1,2.36,89.9,79.9,85.21
2024-02-29 00:12:00,2.0,0.25,15541.54,15821.29,0.2545,60.9,47000,16.2,0.002,124.33,167.85,0.0027,13.1,62166.15,62665.47,"""15MAR24""",0.041986,0.1,2.36,87.44,78.15,82.68
2024-02-29 00:12:00,2.0,0.2345,14577.96,14857.71,0.239,60.9,48000,41.9,0.0024,149.2,198.93,0.0032,12.3,62166.15,62665.47,"""15MAR24""",0.041986,0.1,1.0,84.61,75.85,80.33
2024-02-29 00:12:00,2.0,0.219,13614.39,13894.13,0.2235,60.9,49000,41.7,0.003,186.5,236.23,0.0038,12.5,62166.15,62665.47,"""15MAR24""",0.041986,0.1,45.15,81.47,74.2,78.04
2024-02-29 00:12:00,2.0,0.2035,12650.81,12961.64,0.2085,60.9,50000,41.4,0.0037,230.01,273.53,0.0044,7.7,62166.15,62665.47,"""15MAR24""",0.041986,0.1,52.41,79.88,72.4,75.32
2024-02-29 00:12:00,2.0,0.1885,11718.32,12029.15,0.1935,60.9,51000,40.1,0.0047,292.18,341.91,0.0055,21.4,62166.15,62665.47,"""15MAR24""",0.041986,0.1,56.9,77.7,71.16,73.99
2024-02-29 00:12:00,2.0,0.174,10816.91,11127.74,0.179,60.9,52000,27.4,0.006,373.0,435.16,0.007,67.4,62166.15,62665.47,"""15MAR24""",0.041986,0.1,59.97,76.41,70.15,73.16
2024-02-29 00:12:00,2.0,0.1595,9915.5,10257.41,0.165,62.2,53000,32.0,0.0075,466.25,528.41,0.0085,42.9,62166.15,62665.47,"""15MAR24""",0.041986,0.1,60.54,75.63,68.89,71.51
